In [1]:
# Make sure autoreload is properly set up for auto reload the module if changes
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))
print(parent_dir)

/home/ct-admin/Documents/Langgraph/00_AudioProject/02_Call_Center_AI_Agent


In [7]:
# Cell 1: Setup and Imports
from os import wait
from langchain_ollama import ChatOllama
from src.Agents.graph_debtor_simulator import *
from src.Agents.call_center_agent.data_parameter_builder import get_client_data_async
from rich import print as rprint
import time
user_id = "83905"
client_data = await get_client_data_async(user_id,force_reload=True)

2025-05-28 12:26:48,343 - INFO - Fetching fresh data asynchronously for user_id: 83905
2025-05-28 12:26:48,343 - INFO - Executing 1 concurrent database calls for user_id: 83905


2025-05-28 12:26:51,270 - INFO - Completed concurrent database calls in 2.93 seconds
2025-05-28 12:26:51,272 - INFO - Successfully loaded async data for user_id: 83905 in 2.93s


In [8]:
client_data

{'user_id': '83905',
 'profile': {'user_name': 'TONG00010',
  'client_info': {'title': 'Mr',
   'first_name': 'Teko',
   'last_name': 'Tongwane',
   'id_number': '83905',
   'passport': '',
   'employer': '',
   'client_full_name': 'Teko Tongwane',
   'date_of_birth': None,
   'email_address': 'dev@onecell.co.za',
   'contact': {'telephone': '000 000 0000',
    'fax': '000 000 0000',
    'mobile': '000 000 0000'}},
  'addresses': [{'type': 'Physical',
    'address_line1': 'House No 1379',
    'address_line2': 'Unit 2',
    'post_code': '8474',
    'province': 'NC'},
   {'type': 'Postal',
    'address_line1': 'P.O Box 3143',
    'address_line2': '',
    'post_code': '8447',
    'province': 'NC'}],
  'vehicles': [{'registration': 'TEMP-CT723744',
    'make': 'Toyota',
    'model': 'Hilux 2.7i D/C',
    'color': 'White',
    'chassis_number': 'AHT33UNK408013741',
    'model_year': 2002,
    'contract_status': 'Cancelled',
    'terminal_serial': None,
    'terminal_last_response': None}],


In [ ]:
# Debtor Simulator Test - Jupyter Notebook

# Cell 1: Setup and Imports
from os import wait
from langchain_ollama import ChatOllama
from src.Agents.graph_debtor_simulator import *
from src.Agents.call_center_agent.data_parameter_builder import get_client_data_async
from rich import print as rprint
import time
user_id = "83906"
client_data = await get_client_data_async(user_id)


def test_debtor_response(debtor_agent, message: str, personality_type: str = "unknown"):
    """Test a single debtor response and display results."""
    print(f"\n📞 TESTING: {personality_type.upper()}")
    print(f"👤 Agent says: {message}")
    
    try:
        response = debtor_agent.invoke({"messages": [("user", message)]})
        debtor_response = response['messages'][-1].content
        print(f"🎭 Debtor responds: {debtor_response}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("-" * 60)


def run_comprehensive_tests(llm, client_data):
    """Run all test scenarios systematically."""
    
    print("🎯 CREATING ALL DEBTOR TYPES...")
    
    # Create all debtor types
    debtors = {
        "cooperative": create_cooperative_debtor(llm, client_data),
        "difficult": create_difficult_debtor(llm, client_data),
        "confused": create_confused_debtor(llm, client_data),
        "busy": create_busy_debtor(llm, client_data),
        "suspicious": create_suspicious_debtor(llm, client_data),
        "wrong_person": create_wrong_person(llm, client_data),
        "spouse": create_third_party_spouse(llm, client_data),
        "parent": create_third_party_parent(llm, client_data),
        "assistant": create_third_party_assistant(llm, client_data),
        "employee": create_third_party_employee(llm, client_data)
    }
    
    # Extract client info
    client_name = client_data['profile']['client_info']['client_full_name']
    client_title = client_data['profile']['client_info']['title']
    outstanding_amount = client_data.get('account_aging', {}).get('xbalance', '399.00')
    
    print(f"✅ Created {len(debtors)} debtor simulators")
    print(f"🎯 Testing with client: {client_title} {client_name}")
    print("=" * 80)
    
    # Test scenarios
    test_scenarios = [
        {
            "name": "Initial Contact",
            "message": f"Good day, may I speak to {client_title} {client_name} please?",
            "test_types": ["cooperative", "difficult", "suspicious", "wrong_person", "spouse", "parent"]
        },
        {
            "name": "Identity Verification",
            "message": "This is John from Cartrack Accounts Department. For security purposes, could you please confirm your ID number?",
            "test_types": ["cooperative", "difficult", "suspicious"]
        },
        {
            "name": "Payment Discussion",
            "message": f"{client_title} {client_name}, you have an outstanding balance of R{outstanding_amount} on your account. Can we arrange payment today?",
            "test_types": ["cooperative", "difficult", "busy"]
        },
        {
            "name": "Wrong Person Persistence",
            "message": f"Are you sure you're not {client_name}? This is regarding a Cartrack account.",
            "test_types": ["wrong_person"]
        },
        {
            "name": "Third Party Message Taking",
            "message": f"Could you please ask {client_name} to call Cartrack urgently at 011 250 3000?",
            "test_types": ["spouse", "parent", "assistant", "employee"]
        },
        {
            "name": "Account Information Request",
            "message": "Can you tell me about the vehicle registration and make of the car?",
            "test_types": ["cooperative", "suspicious", "confused"]
        },
        {
            "name": "Payment Urgency",
            "message": "If we don't receive payment today, the account will be handed to our legal department.",
            "test_types": ["cooperative", "difficult", "busy"]
        },
        {
            "name": "Caller Verification",
            "message": "Who is this? What company are you calling from?",
            "test_types": ["suspicious", "wrong_person", "spouse"]
        }
    ]
    
    # Run all test scenarios
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n🧪 TEST SCENARIO {i}: {scenario['name']}")
        print("=" * 80)
        
        for debtor_type in scenario['test_types']:
            if debtor_type in debtors:
                test_debtor_response(debtors[debtor_type], scenario['message'], debtor_type)
                time.sleep(0.5)  # Small delay between tests
        
        print("\n" + "🔄" * 40 + " SCENARIO COMPLETE " + "🔄" * 40)


def run_quick_tests(llm, client_data):
    """Run essential tests quickly."""
    
    print("⚡ QUICK TEST SUITE")
    print("=" * 50)
    
    # Create key debtor types
    cooperative = create_cooperative_debtor(llm, client_data)
    wrong_person = create_wrong_person(llm, client_data)
    spouse = create_third_party_spouse(llm, client_data)
    
    client_name = client_data['profile']['client_info']['client_full_name']
    client_title = client_data['profile']['client_info']['title']
    
    # Essential tests
    tests = [
        (cooperative, f"Good day, may I speak to {client_title} {client_name} please?", "cooperative"),
        (wrong_person, f"Good day, may I speak to {client_title} {client_name} please?", "wrong_person"),
        (spouse, f"Good day, may I speak to {client_title} {client_name} please?", "spouse"),
        (cooperative, "Could you please confirm your ID number for security?", "cooperative"),
        (wrong_person, f"Are you sure you're not {client_name}?", "wrong_person"),
        (spouse, f"Could you ask {client_name} to call us back?", "spouse")
    ]
    
    for debtor, message, personality in tests:
        test_debtor_response(debtor, message, personality)


def run_specific_scenario_tests(llm, client_data):
    """Test specific challenging scenarios."""
    
    print("🎯 SPECIFIC SCENARIO TESTS")
    print("=" * 50)
    
    client_name = client_data['profile']['client_info']['client_full_name']
    client_title = client_data['profile']['client_info']['title']
    
    # Create specific debtors for focused testing
    scenarios = {
        "Persistent Wrong Person": {
            "debtor": create_wrong_person(llm, client_data),
            "messages": [
                f"May I speak to {client_title} {client_name}?",
                f"This is regarding {client_name}'s Cartrack account.",
                f"Are you sure you don't know {client_name}?",
                "This number is listed for this account."
            ]
        },
        "Protective Spouse": {
            "debtor": create_third_party_spouse(llm, client_data),
            "messages": [
                f"Is {client_name} available?",
                "This is regarding their account with Cartrack.",
                "When would be a good time to reach them?",
                "Is everything okay with the account?"
            ]
        },
        "Suspicious Verification": {
            "debtor": create_suspicious_debtor(llm, client_data),
            "messages": [
                f"Good day {client_title} {client_name}.",
                "This is John from Cartrack Accounts.",
                "I need to verify your identity first.",
                "Could you confirm your ID number?"
            ]
        },
        "Busy Professional": {
            "debtor": create_busy_debtor(llm, client_data),
            "messages": [
                f"Hello {client_title} {client_name}.",
                "This is regarding your Cartrack account.",
                "You have an outstanding balance.",
                "Can we resolve this quickly?"
            ]
        }
    }
    
    for scenario_name, scenario_data in scenarios.items():
        print(f"\n🎬 SCENARIO: {scenario_name}")
        print("-" * 60)
        
        for i, message in enumerate(scenario_data["messages"], 1):
            print(f"\n📞 Turn {i}")
            test_debtor_response(scenario_data["debtor"], message, scenario_name.lower().replace(" ", "_"))


def test_edge_cases(llm, client_data):
    """Test edge cases and unusual scenarios."""
    
    print("🚨 EDGE CASE TESTS")
    print("=" * 50)
    
    # Create edge case debtors
    confused = create_confused_debtor(llm, client_data)
    difficult = create_difficult_debtor(llm, client_data)
    employee = create_third_party_employee(llm, client_data)
    
    client_name = client_data['profile']['client_info']['client_full_name']
    
    edge_cases = [
        {
            "name": "Confused About Everything",
            "debtor": confused,
            "message": "You owe R399 on your Cartrack account. Do you remember signing up?"
        },
        {
            "name": "Aggressive Refusal",
            "debtor": difficult,
            "message": "If you don't pay today, we'll take legal action."
        },
        {
            "name": "Professional Boundary",
            "debtor": employee,
            "message": f"Can you tell {client_name} their car will be repossessed?"
        },
        {
            "name": "Information Fishing",
            "debtor": confused,
            "message": "What's your relationship to the account holder?"
        },
        {
            "name": "Payment Pressure",
            "debtor": difficult,
            "message": "We need R399 immediately or your vehicle tracking will be suspended."
        }
    ]
    
    for case in edge_cases:
        print(f"\n⚠️  EDGE CASE: {case['name']}")
        test_debtor_response(case['debtor'], case['message'], case['name'].lower().replace(" ", "_"))


def run_validation_tests(llm, client_data):
    """Validate that debtors behave according to their personality."""
    
    print("✅ VALIDATION TESTS")
    print("=" * 50)
    
    # Test personality consistency
    validation_message = "Could you please confirm your full name and ID number?"
    
    validators = [
        ("cooperative", create_cooperative_debtor(llm, client_data), "Should provide information readily"),
        ("difficult", create_difficult_debtor(llm, client_data), "Should resist or question request"),
        ("suspicious", create_suspicious_debtor(llm, client_data), "Should ask why information is needed"),
        ("wrong_person", create_wrong_person(llm, client_data), "Should deny being the person"),
        ("spouse", create_third_party_spouse(llm, client_data), "Should not provide debtor's information"),
        ("parent", create_third_party_parent(llm, client_data), "Should not provide debtor's information")
    ]
    
    for personality, debtor, expected_behavior in validators:
        print(f"\n🔍 VALIDATING: {personality.upper()}")
        print(f"Expected: {expected_behavior}")
        test_debtor_response(debtor, validation_message, personality)


# Main test execution function
def execute_test_suite(test_type: str = "comprehensive"):
    """Execute the specified test suite."""
    
    # Initialize LLM
    llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.7)
    
    
    print("🎭 DEBTOR SIMULATOR TEST SUITE")
    print("=" * 80)
    print(f"🤖 Using model: qwen2.5:7b-instruct")
    print(f"👤 Testing with: {client_data['profile']['client_info']['client_full_name']}")
    print("=" * 80)
    
    # Run selected test type
    if test_type == "quick":
        run_quick_tests(llm, client_data)
    elif test_type == "comprehensive":
        run_comprehensive_tests(llm, client_data)
    elif test_type == "scenarios":
        run_specific_scenario_tests(llm, client_data)
    elif test_type == "edge":
        test_edge_cases(llm, client_data)
    elif test_type == "validation":
        run_validation_tests(llm, client_data)
    elif test_type == "all":
        run_quick_tests(llm, client_data)
        run_comprehensive_tests(llm, client_data)
        run_specific_scenario_tests(llm, client_data)
        test_edge_cases(llm, client_data)
        run_validation_tests(llm, client_data)
    else:
        print("❌ Invalid test type. Choose: quick, comprehensive, scenarios, edge, validation, or all")


# Simple usage examples for Jupyter notebooks
def notebook_test_examples():
    """Simple examples for notebook testing."""
    
    print("📝 NOTEBOOK TEST EXAMPLES")
    print("=" * 50)
    
    examples = [
        {
            "title": "Cell 1: Setup",
            "code": """
# Initialize
from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.7)

# Your client data
client_data = {...}  # Your actual client data

# Create debtors
cooperative_debtor = create_cooperative_debtor(llm, client_data)
difficult_debtor = create_difficult_debtor(llm, client_data)
suspicious_debtor = create_suspicious_debtor(llm, client_data)
wrong_person = create_wrong_person(llm, client_data)
spouse = create_third_party_spouse(llm, client_data)
"""
        },
        {
            "title": "Cell 2: Initial Contact Test",
            "code": """
client_name = client_data['profile']['client_info']['client_full_name']
client_title = client_data['profile']['client_info']['title']
message1 = f"Good day, may I speak to {client_title} {client_name} please?"

test_debtor_response(cooperative_debtor, message1, "cooperative")
test_debtor_response(difficult_debtor, message1, "difficult") 
test_debtor_response(suspicious_debtor, message1, "suspicious")
test_debtor_response(wrong_person, message1, "wrong_person")
test_debtor_response(spouse, message1, "spouse")
"""
        },
        {
            "title": "Cell 3: Identity Verification Test",
            "code": """
message2 = "This is John from Cartrack Accounts Department. For security purposes, could you please confirm your ID number?"

test_debtor_response(cooperative_debtor, message2, "cooperative")
test_debtor_response(difficult_debtor, message2, "difficult")
test_debtor_response(suspicious_debtor, message2, "suspicious")
"""
        },
        {
            "title": "Cell 4: Payment Discussion Test",
            "code": """
outstanding = client_data.get('account_aging', {}).get('xbalance', '399.00')
message3 = f"{client_title} {client_name}, you have an outstanding balance of R{outstanding} on your account. Can we arrange payment today?"

test_debtor_response(cooperative_debtor, message3, "cooperative")
test_debtor_response(difficult_debtor, message3, "difficult")
test_debtor_response(suspicious_debtor, message3, "suspicious")
"""
        },
        {
            "title": "Cell 5: Wrong Person Persistence Test",
            "code": """
message4 = f"Are you sure you're not {client_name}? This is regarding a Cartrack vehicle tracking account."

test_debtor_response(wrong_person, message4, "wrong_person")
"""
        },
        {
            "title": "Cell 6: Third Party Message Test",
            "code": """
message5 = f"Could you please ask {client_name} to call Cartrack urgently at 011 250 3000 regarding their account?"

test_debtor_response(spouse, message5, "spouse")
"""
        },
        {
            "title": "Cell 7: Quick Comprehensive Test",
            "code": """
# Run all essential tests quickly
execute_test_suite("quick")
"""
        }
    ]
    
    for example in examples:
        print(f"\n{example['title']}")
        print("-" * 40)
        print(example['code'])


if __name__ == "__main__":
    # Example usage
    print("🎯 Select test type:")
    print("1. quick - Essential tests only")
    print("2. comprehensive - All scenarios")
    print("3. scenarios - Specific challenging scenarios")
    print("4. edge - Edge cases and unusual situations")
    print("5. validation - Personality consistency checks")
    print("6. all - Everything")
    print("7. examples - Show notebook examples")
    
    choice = input("\nEnter choice (1-7): ").strip()
    
    test_map = {
        "1": "quick",
        "2": "comprehensive", 
        "3": "scenarios",
        "4": "edge",
        "5": "validation",
        "6": "all",
        "7": "examples"
    }
    
    if choice == "7":
        notebook_test_examples()
    elif choice in test_map:
        execute_test_suite(test_map[choice])
    else:
        print("❌ Invalid choice")
        execute_test_suite("quick")  # Default to quick tests



In [ ]:
# Cell 1: Setup
from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.7)

cooperative_debtor = create_cooperative_debtor(llm, client_data)
wrong_person = create_wrong_person(llm, client_data)
spouse = create_third_party_spouse(llm, client_data)

# Cell 2: Test Initial Contact
client_name = client_data['profile']['client_info']['client_full_name'] 
client_title = client_data['profile']['client_info']['title']
message1 = f"Good day, may I speak to {client_title} {client_name} please?"

test_debtor_response(cooperative_debtor, message1, "cooperative")
test_debtor_response(wrong_person, message1, "wrong_person")
test_debtor_response(spouse, message1, "spouse")

# Cell 3: Test Identity Verification  
message2 = "For security, could you please confirm your ID number?"
test_debtor_response(cooperative_debtor, message2, "cooperative")

# Cell 4: Test Payment Discussion
message3 = f"You have R399.00 outstanding. Can we arrange payment today?"
test_debtor_response(cooperative_debtor, message3, "cooperative")

In [11]:
client_data

{'user_id': '83906',
 'profile': {'user_name': 'OELO00016',
  'client_info': {'title': 'Mrs',
   'first_name': 'Linda',
   'last_name': 'Oelofse',
   'id_number': '83906',
   'passport': '',
   'employer': '',
   'client_full_name': 'Linda Oelofse',
   'date_of_birth': None,
   'email_address': 'dev@onecell.co.za',
   'contact': {'telephone': '000 000 00000',
    'fax': '000 000 0000',
    'mobile': '000 000 0000'}},
  'addresses': [{'type': 'Physical',
    'address_line1': 'P O BOX 15076',
    'address_line2': 'Lambton',
    'post_code': '1414',
    'province': 'Gauteng'},
   {'type': 'Postal',
    'address_line1': 'P O BOX 15076',
    'address_line2': 'Lambton',
    'post_code': '1414',
    'province': 'Gauteng'}],
  'vehicles': [{'registration': 'JY96NMGP',
    'make': 'Toyota',
    'model': 'Hilux',
    'color': 'White',
    'chassis_number': 'AHTJB3DD3045226941MN',
    'model_year': 2021,
    'contract_status': 'Active',
    'terminal_serial': 'BA002202',
    'terminal_last_respon